<a href="https://colab.research.google.com/github/thomasdotred/Artificial_Inteligence-/blob/master/MNIST_example_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import TF and MNIST

In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.keras.callbacks import TensorBoard

In [0]:
mnist = input_data.read_data_sets('MNIST_data', one_hot =True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [0]:
!nvidia-smi

Wed Apr 10 10:07:54 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    26W / 149W |      1MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Helper functions

## initalizer functions

In [0]:
# init weights
def init_weights(shape):
  init_random_dist = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(init_random_dist)


In [0]:
# init bias
def init_bias(shape):
  init_bias_vals = tf.constant(0.1, shape=[shape])
  return tf.Variable(init_bias_vals)

## layer functions

In [0]:
# Conv2d
def conv2d(x,w):
  # x ---> [batch, H, W, Chan]
  # w ---> [filter H, filter W, Channels IN, Channels OUT]
  return tf.nn.conv2d(x, w, strides=[1,1,1,1], padding='SAME', use_cudnn_on_gpu=True)

In [0]:
# POOLING
def max_pool_2by2(x):
  #x ---> [batch,h, w, c]
  # only poolig in h and w so [1,2,2,1]
  return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [0]:
# Conv layer
def conv_layer(input_x, shape):
  w = init_weights(shape)
  b = init_bias(shape[3])
  return tf.nn.relu(conv2d(input_x, w) + b)

In [0]:
# Fully connected layer
def normal_full_layer(input_layer, size):
  input_size = int(input_layer.get_shape()[1])
  w = init_weights([input_size, size])
  b = init_bias(size)
  return tf.matmul(input_layer, w) + b

# Network

## Prepare

In [0]:
# Placeholders
x = tf.placeholder(tf.float32, shape=[None, 784]) 
y_true = tf.placeholder(tf.float32, shape=[None, 10])
x_image = tf.reshape(x, [-1, 28, 28, 1])

## Network layer design

In [0]:
# Layers

convo_1 = conv_layer(x_image, shape=[5,5,1,32])
convo_1_pool = max_pool_2by2(convo_1)

convo_2 = conv_layer(convo_1_pool, shape = [5,5,32,64])
convo_2_pool = max_pool_2by2(convo_2)

convo_2_flat = tf.reshape(convo_2_pool, [-1, 7*7*64])
full_layer_1 = tf.nn.relu(normal_full_layer(convo_2_flat, 1024))

# Dropout 
hold_prob = tf.placeholder(tf.float32)
full_1_dropout = tf.nn.dropout(full_layer_1, rate=hold_prob)

y_pred = normal_full_layer(full_1_dropout, 10)

Instructions for updating:
Colocations handled automatically by placer.


## Network training methods

In [0]:
# Loss fuction
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true, logits=y_pred))

# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

# Initalize and run session

In [0]:
init = tf.global_variables_initializer()
epochs = 500

with tf.Session() as sess:
  
  writer = tf.summary.FileWriter( './logs/1/train ', sess.graph)
  
  for i in range(epochs):
    sess.run(init)

    batch_x, batch_y = mnist.train.next_batch(50)
    feed = {
        x:batch_x,
        y_true:batch_y,
        hold_prob:0.5
    }
    sess.run(train, feed_dict=feed)
    
    if i%100 ==0:
      print('on step: {}'.format(i))
      print('Accuracy: ')
      matches = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true,1))
      acc = tf.reduce_mean(tf.cast(matches, tf.float32))
      
      
      val_feed = {
        x:mnist.test.images,
        y_true:mnist.test.labels,
        hold_prob:1.0
      }
      print(sess.run(acc,feed_dict=val_feed))
      print('\n')
  writer.close()

on step: 0
Accuracy: 
0.098


on step: 100
Accuracy: 
0.098


on step: 200
Accuracy: 
0.098


on step: 300
Accuracy: 
0.098


on step: 400
Accuracy: 
0.098


